In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
import sys
import os

sys.path.append("..")
from model import Net


In [4]:
def get_model_params(model):
    total_params = sum(p.numel() for p in model.parameters())
    print(f'Total Parameters: {total_params}')
    return total_params

model = Net()
get_model_params(model)


Total Parameters: 4594


4594

In [5]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
         Dropout2d-3            [-1, 8, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           1,168
       BatchNorm2d-5           [-1, 16, 28, 28]              32
         Dropout2d-6           [-1, 16, 28, 28]               0
         MaxPool2d-7           [-1, 16, 14, 14]               0
            Conv2d-8            [-1, 4, 14, 14]              68
            Conv2d-9            [-1, 8, 14, 14]             296
      BatchNorm2d-10            [-1, 8, 14, 14]              16
        Dropout2d-11            [-1, 8, 14, 14]               0
           Conv2d-12           [-1, 16, 14, 14]           1,168
      BatchNorm2d-13           [-1, 16, 14, 14]              32
        Dropout2d-14           [-1, 16,

In [6]:


torch.manual_seed(1)
batch_size = 64

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
    transforms.RandomRotation(10),  # Rotate by up to 10 degrees
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Translate by up to 10% in each direction
    transforms.RandomPerspective(distortion_scale=0.5, p=0.5),  # Apply perspective transform
])

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=transform),
    batch_size=batch_size, 
    shuffle=True, 
    **kwargs
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transform),
    batch_size=batch_size, 
    shuffle=True, 
    **kwargs
)


In [7]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        accuracy = (output.argmax(dim=1) == target).float().mean()
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'batch_id={batch_idx}; loss={loss.item():.4f}; accuracy={accuracy.item():.3f}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
epochs = 20

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

batch_id=937; loss=0.3632; accuracy=0.969: 100%|██████████| 938/938 [01:37<00:00,  9.61it/s]


Test set: Average loss: 0.4023, Accuracy: 8843/10000 (88%)



batch_id=937; loss=0.4376; accuracy=0.781: 100%|██████████| 938/938 [01:07<00:00, 13.83it/s]


Test set: Average loss: 0.2571, Accuracy: 9158/10000 (92%)



batch_id=937; loss=0.5886; accuracy=0.812: 100%|██████████| 938/938 [01:07<00:00, 13.96it/s]


...

batch_id=937; loss=0.2032; accuracy=0.938: 100%|██████████| 938/938 [01:11<00:00, 13.15it/s]


Test set: Average loss: 0.1974, Accuracy: 9363/10000 (94%)



batch_id=937; loss=0.5388; accuracy=0.781: 100%|██████████| 938/938 [02:06<00:00,  7.39it/s]


Test set: Average loss: 0.1712, Accuracy: 9473/10000 (95%)



batch_id=368; loss=0.3313; accuracy=0.906:  39%|███▉      | 369/938 [00:29<00:39, 14.43it/s]